In [6]:
pip install gurobipy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\himan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from gurobipy import Model
model = Model()
print("Gurobi is installed and working!")


Restricted license - for non-production use only - expires 2026-11-23
Gurobi is installed and working!


In [8]:
print(f"cp type: {type(cp)}")

cp type: <class 'float'>


In [9]:
cp = 3.14  # This would overwrite cvxpy import

In [10]:
import cvxpy as cp

print(f"cp type: {type(cp)}")  # Should show <class 'cvxpy.__init__.py'>


cp type: <class 'module'>


In [11]:
import cvxpy as cp
import numpy as np

# Define the number of variables (n)
n = 10

# Create cvxpy variables
x = cp.Variable(n)
Tbuffer = cp.Variable(n)

# Example constraint
constraints = [x >= 0, x <= 10]  # x should be between 0 and 10

# Example objective function
objective = cp.Minimize(cp.sum(x))

# Define the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the result
print(f"Optimal value of x: {x.value}")

Optimal value of x: [3.04802017e-10 3.04802017e-10 3.04802017e-10 3.04802017e-10
 3.04802017e-10 3.04802017e-10 3.04802017e-10 3.04802017e-10
 3.04802017e-10 3.04802017e-10]


In [12]:
import pandas as pd
import numpy as np
import cvxpy as cp  # Ensure cvxpy is imported correctly
import time
# Import everything from BasicDefinitions.py (adjust as needed)
from BasicDefinitions import *

# Read Data file - one year
df = pd.read_csv("YearHourly.csv", header=None, index_col=0)
df.columns = ["Tout", "Qhouse", "Price el."]
df.index.name = "Time"
df.info()

Tout = df["Tout"].to_numpy()
Tflow = fTflow(Tout)

Qhouse = df["Qhouse"].to_numpy()
p = df["Price el."]
n = p.size

# Elemenatary matrices, like in Octave BasicDefinitions.m
E = np.eye(n)
Z = np.zeros_like(E)
S = np.tril(np.ones_like(E), 0)

# Matrices for the linear problem, see report
M = S / C
r = Tinit - np.matmul(S, Qhouse) / C

# Equality constraint for calculating Tbuffer
AE = np.hstack((-M, E))
bE = r

# Produce enough energy (Upper bound)
A_up = -np.hstack((np.ones(shape=(1, n)), np.zeros(shape=(1, n))))
b_up = -Qhouse.sum()

# Cost vector 
a = 3
cel = p / CoP(Tout, Tflow)
cte = p / CoP(Tout, Tflow + 1) - p / CoP(Tout, Tflow)
c = np.hstack((cel, a * cte))

 #Bounds, x in [0,1], Tbuffer in [Tflow, Tmax]
bounds = [(0, Pth)] * n + [(Tflow[i].item(), Tmax) for i in range(n)]

# Create cvxpy variables for optimization
x = cp.Variable(n)  # Ensure cvxpy is being used here
Tbuffer = cp.Variable(n)

# Define the cost function and constraints
cost = cp.sum(c * x)  # Assuming the cost is linear in terms of x
constraints = [
    AE @ cp.hstack([x, Tbuffer]) == bE,  # Equality constraint
    A_up @ cp.hstack([x, Tbuffer]) <= b_up,  # Upper bound energy constraint
    x >= 0, x <= 1,  # Bounds on x
    Tbuffer >= Tflow, Tbuffer <= Tmax  # Bounds on Tbuffer
]

# Create problem
problem = cp.Problem(cp.Minimize(cost), constraints)

# Solve the problem using different solvers
start_time = time.time()

# Choose solver for performance
solver = cp.ECOS  # You can replace with other solvers like cp.SCS, cp.GUROBI, cp.MOSEK
problem.solve(solver=solver)

# Output results
end_time = time.time()
print(f"Optimization completed in {end_time - start_time:.2f} seconds using {solver}")
print(f"Optimal solution for x: {x.value}")
print(f"Optimal solution for Tbuffer: {Tbuffer.value}")

# Reference Costs for demand-driven solution
print(f"Costs demand-driven solution {cel.dot(Qhouse):.2f} €")


<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 2021-01-01 00:00:00 to 2021-12-31 23:00:00
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Tout       8760 non-null   float64
 1   Qhouse     8760 non-null   float64
 2   Price el.  8760 non-null   float64
dtypes: float64(3)
memory usage: 273.8+ KB


AttributeError: 'float' object has no attribute 'Variable'

In [ ]:
print(f"cp type: {type(cp)}")


cp type: <class 'float'>


In [ ]:
cp = 3.14  # This would overwrite cvxpy import


In [ ]:
import cvxpy as cp

print(f"cp type: {type(cp)}")  # Should show <class 'cvxpy.__init__.py'>


cp type: <class 'module'>


In [ ]:
import cvxpy as cp
import numpy as np

# Define the number of variables (n)
n = 10

# Create cvxpy variables
x = cp.Variable(n)
Tbuffer = cp.Variable(n)

# Example constraint
constraints = [x >= 0, x <= 10]  # x should be between 0 and 10

# Example objective function
objective = cp.Minimize(cp.sum(x))

# Define the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the result
print(f"Optimal value of x: {x.value}")


Optimal value of x: [3.04802017e-10 3.04802017e-10 3.04802017e-10 3.04802017e-10
 3.04802017e-10 3.04802017e-10 3.04802017e-10 3.04802017e-10
 3.04802017e-10 3.04802017e-10]


In [ ]:
cost = cp.sum(price_el * P_hp / CoP(Tout, T_buffer))  # Total energy cost


C:\Users\himan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 8 times so far.

  warnings.warn(msg, UserWarning)


RuntimeError: 
You're calling a NumPy function on a CVXPY expression. This is prone to causing
errors or code that doesn't behave as expected. Consider using one of the
functions documented here: https://www.cvxpy.org/tutorial/functions/index.html


In [ ]:
pip install cvxpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: C:\Users\himan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import cvxpy as cp

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import cvxpy as cp  # Ensure this import is correct
import time
from BasicDefinitions import *

# Read Data file - one year
df = pd.read_csv("YearHourly.csv", header=None, index_col=0)
df.columns = ["Tout", "Qhouse", "Price el."]
df.index.name = "Time"

# Convert columns to numpy arrays
Tout = df["Tout"].to_numpy()
Tflow = fTflow(Tout)
Qhouse = df["Qhouse"].to_numpy()
p = df["Price el."].to_numpy()
n = p.size

# Elementary matrices
E = np.eye(n)
Z = np.zeros_like(E)
S = np.tril(np.ones_like(E), 0)

# Matrices for the linear problem
M = S / C 
r = Tinit - np.matmul(S, Qhouse) / C

# Define variables
Qprod = cp.Variable(n)  # Production of heat
Tbuffer = cp.Variable(n)  # Buffer temperature

# Cost vector
a = 3
cel = p / CoP(Tout, Tflow)
cte = p / CoP(Tout, Tflow + 1) - p / CoP(Tout, Tflow)
c = cp.hstack([cel, a * cte])

# Objective function
objective = cp.Minimize(c @ cp.hstack([Qprod, Tbuffer]))

# Constraints
constraints = [
    -M @ Qprod + Tbuffer == r,  # Equality constraint for calculating Tbuffer
    cp.sum(Qprod) >= Qhouse.sum(),  # Produce enough energy
    0 <= Qprod, Qprod <= Pth,  # Bounds for Qprod
    Tflow <= Tbuffer, Tbuffer <= Tmax  # Bounds for Tbuffer
]

# Solve the problem
problem = cp.Problem(objective, constraints)
start = time.time()
problem.solve(solver=cp.ECOS)  # You can change the solver to CPLEX, GUROBI, etc., if available
end = time.time()

# Print results
print(f"Runtime linear solver {end - start:.2f} s")
print(f"Costs including temperature costs {problem.value:.2f} €")
print(f"Costs excluding temperature costs {cel @ Qprod.value:.2f} €")

# Extract data for later use
Qprod_opt = Qprod.value
Tbuffer_opt = Tbuffer.value

# Show complete result
print("Optimal Qprod:", Qprod_opt)
print("Optimal Tbuffer:", Tbuffer_opt)

AttributeError: 'float' object has no attribute 'Variable'

In [ ]:
n = p.size  # This should be an integer

In [ ]:
print(type(cp))  # Should output <class 'module'>

<class 'float'>


In [ ]:
print(n)  # Should output an integer

8760


In [ ]:

import os

In [ ]:
os.environ["PATH"] += os.pathsep + "C:/glpk/bin"

In [ ]:
import cvxpy as cp


In [ ]:
cp = ...
cp = 3.0
print(cp)

3.0


In [1]:
import numpy as np
import pandas as pd

from BasicDefinitions import *

# Load data
df = pd.read_csv("YearHourly.csv", header=None, index_col=0)
df.columns = ["Tout", "Qhouse", "Price el."]
df.index.name = "Time"

Tout = df["Tout"].to_numpy()
Tflow = fTflow(Tout)
Qhouse = df["Qhouse"].to_numpy()
p = df["Price el."].to_numpy()
n = p.size

# Cost coefficients
a = 3
cel = p / CoP(Tout, Tflow)
cte = p / CoP(Tout, Tflow + 1) - p / CoP(Tout, Tflow)

# CVXPY variables
Qprod = cp.Variable(n)
Tbuffer = cp.Variable(n)

# Objective function
objective = cp.Minimize(cel @ Qprod + a * cp.sum(cte * (Tbuffer - Tflow)))

# Constraints
constraints = [
    AE @ cp.hstack([Qprod, Tbuffer]) == bE,
    A_up @ Qprod <= b_up,
    Qprod >= 0,
    Qprod <= Pth,
    Tbuffer >= Tflow,
    Tbuffer <= Tmax
]

# Solve problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GLPK)

# Evaluate solution
Qprod_solution = Qprod.value
Tbuffer_solution = Tbuffer.value

evaluate_solution(Qprod_solution, "CVXPY - GLPK")


AttributeError: 'float' object has no attribute 'Variable'

In [1]:
import cvxpy as cp  


(CVXPY) Mar 20 12:09:29 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 20 12:09:29 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
cp = ...
print(cp)

Ellipsis


In [ ]:
import cvxpy as cvxp  # Rename the import
qprod = cvxp.Variable(n, nonneg=True)  # Now it works


NameError: name 'n' is not defined

## i m not abel to solve this flot vaule 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp  # t
import time
from BasicDefinitions import *

# Load data
df = pd.read_csv("YearHourly.csv", header=None, index_col=0)
df.columns = ["Tout", "Qhouse", "Price el."]
df.index.name = "Time"

tout = df["Tout"].to_numpy()
tflow = fTflow(tout)
qhouse = df["Qhouse"].to_numpy()
p = df["Price el."].to_numpy()
n = p.size

# Define variables
qprod = cp.Variable(n, nonneg=True)  # 
tbuffer = cp.Variable(n)

# Cost coefficients
a = 3
cel = p / CoP(tout, tflow)
cte = p / CoP(tout, tflow + 1) - p / CoP(tout, tflow)

# Objective function
objective = cp.Minimize(cp.sum(cel @ qprod + a * cp.sum(cte * (tbuffer - tflow))))

# Constraints
constraints = []
E = np.eye(n)
S = np.tril(np.ones_like(E), 0)
C = 4.2 * 500 / 3600  # Buffer tank heat capacity in kWh/K
Tinit = 30
M = S / C
r = Tinit - np.matmul(S, qhouse) / C

for i in range(n):
    constraints.append(tbuffer[i] >= tflow[i])
    constraints.append(tbuffer[i] <= Tmax)
    constraints.append(-cp.sum(S[i, :] / C * qprod) + tbuffer[i] == r[i])
constraints.append(cp.sum(qprod) >= qhouse.sum())

# Solve problem
problem = cp.Problem(objective, constraints)
start = time.time()
problem.solve(solver=cp.GLPK)  # 
end = time.time()

# Extract solution
qprod_opt = qprod.value

# Output results
print(f"Runtime linear solver: {end - start:.2f} s")
print(f"Optimal total cost (with temperature costs): {problem.value:.2f} €")
costs_excluding_temperature = np.sum(cel * qprod_opt)
print(f"Costs excluding temperature costs: {costs_excluding_temperature:.2f} €")

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(range(n), qprod_opt, marker='o', linestyle='-', color='b', label='Optimized Qprod')
plt.axhline(y=np.mean(qprod_opt), color='r', linestyle='--', label='Mean Qprod')
plt.xlabel("Time Step")
plt.ylabel("Optimized Qprod")
plt.title("Optimization Results Over Time - CVXPY")
plt.legend()
plt.grid()
plt.show()


AttributeError: 'float' object has no attribute 'Variable'